In [2]:
import numpy as np
import collections

import os
import pickle

import tensorflow as tf

C:\Users\MAIN\Anaconda3\envs\kaggle\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\MAIN\Anaconda3\envs\kaggle\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\MAIN\Anaconda3\envs\kaggle\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\MAIN\Anaconda3\envs\kaggle\lib\site-packages\tensorflow\python\f

In [8]:
# -*- coding: utf-8 -*-

import codecs
import os
import collections
from six.moves import cPickle
import numpy as np


class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        # 전처리된 파일들("vocab.pkl", "data.npy")이 이미 존재하면 이를 불러오고 없으면 데이터 전처리를 진행합니다.
        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        # 배치를 생성하고 배치 포인터를 배치의 시작지점으로 리셋합니다.
        self.create_batches()
        self.reset_batch_pointer()

    # 데이터 전처리를 진행합니다.
    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding=self.encoding) as f:
            data = f.read()
        # 데이터에서 문자(character)별 등장횟수를 셉니다.
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs) # 전체 문자들(Chracters)
        self.vocab_size = len(self.chars) # 전체 문자(단어) 개수
        self.vocab = dict(zip(self.chars, range(len(self.chars)))) # 단어들을 (charcter, id) 형태의 dictionary로 만듭니다.
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        # 데이터의 각각의 character들을 id로 변경합니다.
        self.tensor = np.array(list(map(self.vocab.get, data)))
        # id로 변경한 데이터를 "data.npy" binary numpy 파일로 저장힙니다.
        np.save(tensor_file, self.tensor)

    # 전처리한 데이터가 파일로 저장되어 있다면 파일로부터 전처리된 정보들을 읽어옵니다.
    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    # 전체 데이터를 배치 단위로 묶습니다.
    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

        # 데이터 양이 너무 적어서 1개의 배치도 만들수없을 경우, 에러 메세지를 출력합니다.
        if self.num_batches == 0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        # 배치에 필요한 정수만큼의 데이터만을 불러옵니다. e.g. 1115394 -> 1115000
        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        # 타겟 데이터는 인풋 데이터를 한칸 뒤로 민 형태로 구성합니다.
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        # batch_size 크기의 배치를 num_batches 개수 만큼 생성합니다. 
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1),
                                  self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1),
                                  self.num_batches, 1)

    # 다음 배치롤 불러오고 배치 포인터를 1만큼 증가시킵니다.  
    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    # 배치의 시작점을 데이터의 시작지점으로 리셋합니다.
    def reset_batch_pointer(self):
        self.pointer = 0

In [10]:
tf.reset_default_graph()
# -*- coding: utf-8 -*-
# Char-RNN 예제
# Author : solaris33
# Project URL : http://solarisailab.com/archives/2487
# GitHub Repository : https://github.com/solaris33/char-rnn-tensorflow/
# Reference : https://github.com/sherjilozair/char-rnn-tensorflow

import tensorflow as tf
import numpy as np

from utils import TextLoader

# 학습에 필요한 설정값들을 지정합니다.
data_dir = 'data/tinyshakespeare' # 셰익스피어 희곡 <리처드 3세> 데이터로 학습
#data_dir = 'data/linux' # <Linux 소스코드> 데이터로 학습
batch_size = 50 # Training : 50, Sampling : 1
seq_length = 50 # Training : 50, Sampling : 1
hidden_size = 128   # 히든 레이어의 노드 개수
learning_rate = 0.002
num_epochs = 2
num_hidden_layers = 2
grad_clip = 5   # Gradient Clipping에 사용할 임계값

# TextLoader를 이용해서 데이터를 불러옵니다.
data_loader = TextLoader(data_dir, batch_size, seq_length)
# 학습데이터에 포함된 모든 단어들을 나타내는 변수인 chars와 chars에 id를 부여해 dict 형태로 만든 vocab을 선언합니다.
chars = data_loader.chars 
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size # 전체 단어개수

# 인풋데이터와 타겟데이터, 배치 사이즈를 입력받기 위한 플레이스홀더를 설정합니다.
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input_data : [batch_size, seq_length])
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target_data : [batch_size, seq_length])
state_batch_size = tf.placeholder(tf.int32, shape=[])      # Training : 50, Sampling : 1

# RNN의 마지막 히든레이어의 출력을 소프트맥스 출력값으로 변환해주기 위한 변수들을 선언합니다.
# hidden_size -> vocab_size
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# num_hidden_layers만큼 LSTM cell(히든레이어)를 선언합니다.
cells = []
for _ in range(0, num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)

# cell을 종합해서 RNN을 정의합니다.
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 Embedding Matrix를 선언합니다.
# vocab_size -> hidden_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화합니다.
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언합니다.
# outputs : [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
# ouputs을 [batch_size * seq_length, hidden_size]] 형태로 바꿉니다.
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정합니다.
# logits : [batch_size * seq_length, vocab_size]
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# Cross Entropy 손실 함수를 정의합니다. 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저를 선언하고 옵티마이저에 Gradient Clipping을 적용합니다.
# grad_clip(=5)보다 큰 Gradient를 5로 Clippin합니다.
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습을 진행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        data_loader.reset_batch_pointer()
        # 초기 상태값을 지정합니다.
        state = sess.run(initial_state, feed_dict={state_batch_size : batch_size})

        for b in range(data_loader.num_batches):
            # x, y 데이터를 불러옵니다.
            x, y = data_loader.next_batch()
            # y에 one_hot 인코딩을 적용합니다. 
            y = tf.one_hot(y, vocab_size)            # y : [batch_size, seq_length, vocab_size]
            y = tf.reshape(y, [-1, vocab_size])       # y : [batch_size * seq_length, vocab_size]
            y = y.eval()

            # feed-dict에 사용할 값들과 LSTM 초기 cell state(feed_dict[c])값과 hidden layer 출력값(feed_dict[h])을 지정합니다.
            feed_dict = {input_data : x, target_data: y, state_batch_size : batch_size}
            for i, (c, h) in enumerate(initial_state):
                feed_dict[c] = state[i].c
                feed_dict[h] = state[i].h

            # 한스텝 학습을 진행합니다.
            _, loss_print, state = sess.run([train_step, loss, final_state], feed_dict=feed_dict)

            print("{}(학습한 배치개수)/{}(학습할 배치개수), 반복(epoch): {}, 손실함수(loss): {:.3f}".format(
                          e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          (e+1), 
                          loss_print))

    print("트레이닝이 끝났습니다!")   
    

    # 샘플링 시작
    print("샘플링을 시작합니다!")
    num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다. 
    prime = u' '         # 시작 글자를 ' '(공백)으로 지정합니다.
    sampling_type = 1    # 샘플링 타입을 설정합니다.
    state = sess.run(cell.zero_state(1, tf.float32)) # RNN의 최초 state값을 0으로 초기화합니다.

    # Random Sampling을 위한 weighted_pick 함수를 정의합니다.
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime       # 샘플링 결과를 리턴받을 ret 변수에 첫번째 글자를 할당합니다.
    char = prime[-1]   # Char-RNN의 첫번쨰 인풋을 지정합니다.  
    for n in range(num_sampling):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]

        # RNN을 한스텝 실행하고 Softmax 행렬을 리턴으로 받습니다.
        feed_dict = {input_data: x, state_batch_size : 1, initial_state: state}
        [probs_result, state] = sess.run([probs, final_state], feed_dict=feed_dict)         

        # 불필요한 차원을 제거합니다.
        # probs_result : (1,65) -> p : (65)
        p = np.squeeze(probs_result)

        # 샘플링 타입에 따라 3가지 종류로 샘플링 합니다.
        # sampling_type : 0 -> 다음 글자를 예측할때 항상 argmax를 사용
        # sampling_type : 1(defualt) -> 다음 글자를 예측할때 항상 random sampling을 사용
        # sampling_type : 2 -> 다음 글자를 예측할때 이전 글자가 ' '(공백)이면 random sampling, 그렇지 않을 경우 argmax를 사용
        if sampling_type == 0:
            sample = np.argmax(p)
        elif sampling_type == 2:
            if char == ' ':
                sample = weighted_pick(p)
            else:
                sample = np.argmax(p)
        else:
            sample = weighted_pick(p)

        pred = chars[sample]
        ret += pred     # 샘플링 결과에 현재 스텝에서 예측한 글자를 추가합니다. (예를들어 pred=L일 경우, ret = HEL -> HELL)
        char = pred     # 예측한 글자를 다음 RNN의 인풋으로 사용합니다.

    print("샘플링 결과:")
    print(ret)

loading preprocessed files
0(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.713
1(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.468
2(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.135
3(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.850
4(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.515
5(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.572
6(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.356
7(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.352
8(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.335
9(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.241
10(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.148
11(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.117
12(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.097
13(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.055
14(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.000
15(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.938
16(학습한 배치개수)/924(학습할 배치

136(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.897
137(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.921
138(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.906
139(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.901
140(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.883
141(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.908
142(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.877
143(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.900
144(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.911
145(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.855
146(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.878
147(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.910
148(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.906
149(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.892
150(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.947
151(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.930
152(학습한 배치개수)/924(학습할 배치

272(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.753
273(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.707
274(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.718
275(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.710
276(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.726
277(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.642
278(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.699
279(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.692
280(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.724
281(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.722
282(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.761
283(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.674
284(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.691
285(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.779
286(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.704
287(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.748
288(학습한 배치개수)/924(학습할 배치

408(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.693
409(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.722
410(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.741
411(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.623
412(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.601
413(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.590
414(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.575
415(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.592
416(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.609
417(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.636
418(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.621
419(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.611
420(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.557
421(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.620
422(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.614
423(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.603
424(학습한 배치개수)/924(학습할 배치

544(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.534
545(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.613
546(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.529
547(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.607
548(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.605
549(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.552
550(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.557
551(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.561
552(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.624
553(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.494
554(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.485
555(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.499
556(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.509
557(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.512
558(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.548
559(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.536
560(학습한 배치개수)/924(학습할 배치

679(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.548
680(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.527
681(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.525
682(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.515
683(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.525
684(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.585
685(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.530
686(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.510
687(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.473
688(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.544
689(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.571
690(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.495
691(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.481
692(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.535
693(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.600
694(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.548
695(학습한 배치개수)/924(학습할 배치

814(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.455
815(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.525
816(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.490
817(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.422
818(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.501
819(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.533
820(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.573
821(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.551
822(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.476
823(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.540
824(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.423
825(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.409
826(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.451
827(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.459
828(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.506
829(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.473
830(학습한 배치개수)/924(학습할 배치

In [11]:
print('hello')

hello
